In [1]:
import sys
import ROOT as R       # Make sure the ROOT system is properly initialized or ROOT will not be found.
R.EnableImplicitMT()   # Note: Comment this if you want to use df.Display().
# Comment the next line this if you don't like the interactive histograms.
%jsroot on  
# Load the MiniDST Library. Not really needed, but can be handy in event by event checking of data and getting constants.


In [2]:
#R.gSystem.Load("/data/HPS/Analyses/ECAL/build/lib/libEcal_Analysis")
#R.gInterpreter.ProcessLine('''auto EAC = Ecal_Analysis_Class();''')   # This is key. It puts the EAC in C++ space.
#print(f"{R.EAC.Version()}")
R.gSystem.Load("/data/HPS/lib/libMiniDst")
MiniDst = R.MiniDst()
print("Version of MiniDst Class:",MiniDst._version_())


Version of MiniDst Class: 1.2.2


In [3]:
data_files_pattern =  "/data/HPS/data/physrun2021/Tritrig/Pure/HPS_Run2021Pass1_v5_3pt74_2x_minidst.root"
Beam_Energy = 3.74
Ecal_Cluster_Time_offset = -63
ch_moller = R.TChain("MiniDST")
ch_moller.Add(data_files_pattern)
n_events = ch_moller.GetEntries()
nf = ch_moller.GetListOfFiles().GetEntries()
df = R.RDataFrame(ch_moller)
print(f"Added {nf} files to the chain, total of {n_events/1e6:7.4f}M events")

Added 1 files to the chain, total of  0.2096M events


In [4]:
# This is a list of the vertex (v0) particle types from LCIO. 
# ParticleNames=[ str(x) for x in MiniDst.ParticleTypeNames]
# This makes a nicely formatted list of all the items in the dataframe. Note that you can just call df.Describe() but that gives a single column.
All_Names = list(df.GetColumnNames())
Store_Names = []
ii=0
remove_names = [] # ["v0_","hodo_","ext_trigger","part_","rf_","svt"]
for n in All_Names:
    s = str(n)
    if not any([s.lower().startswith(x) for x in remove_names]):  # Allow for a filter.
        Store_Names.append(s)                                     # Store_Names can be handy if you later want to write out the filtered data.
        dat_type = df.GetColumnType(s).replace("ROOT::VecOps::RVec<","RVec<")
        print(f"{s:<30s} [{dat_type:<20s}]", end = " ")
        ii += 1
        if ii%2==0:     # Replace the 2 with 3 to get 3 columns (too wide for my screen)
            print("");


ecal_cluster_energy            [RVec<double>        ] ecal_cluster_hits              [RVec<vector<int>>   ] 
ecal_cluster_mc_id             [RVec<int>           ] ecal_cluster_mc_pdg            [RVec<int>           ] 
ecal_cluster_mc_pdg_purity     [RVec<double>        ] ecal_cluster_nhits             [RVec<int>           ] 
ecal_cluster_seed_energy       [RVec<double>        ] ecal_cluster_seed_index        [RVec<int>           ] 
ecal_cluster_seed_ix           [RVec<int>           ] ecal_cluster_seed_iy           [RVec<int>           ] 
ecal_cluster_time              [RVec<double>        ] ecal_cluster_uncor_energy      [RVec<double>        ] 
ecal_cluster_uncor_hits        [RVec<vector<int>>   ] ecal_cluster_uncor_nhits       [RVec<int>           ] 
ecal_cluster_uncor_seed_energy [RVec<double>        ] ecal_cluster_uncor_seed_index  [RVec<int>           ] 
ecal_cluster_uncor_seed_ix     [RVec<int>           ] ecal_cluster_uncor_seed_iy     [RVec<int>           ] 
ecal_cluster_uncor_

## Trident Selection
These cuts are from Sarah Gaiser's talk at the June 3 2025 collaboration meeting. 
Ref: 
* [Preselection: N-1 Cut Analysis](https://indico.jlab.org/event/965/contributions/16777/attachments/12762/20454/Nm1_cut_analysis_2025_05_20.pdf) (Sarah Gaiser 5/21/2025)
* [Vertex Pre-selection](https://indico.jlab.org/event/965/contributions/16777/attachments/12762/20448/preselection_data_MC_sarah.pdf) (Sarah Gaiser, June Collab meeting, 6/3/2025) 
* [Comparing skimmed and no-skim
datasets](https://indico.slac.stanford.edu/event/9638/contributions/12541/attachments/5498/14137/hps_meeting_2025_04_22_sarah.pdf) (Sarah Gaiser 4/22/2025)
* [First Look at Tight Selection](https://indico.slac.stanford.edu/event/9634/contributions/12314/attachments/5410/13985/hps_meeting_2025_03_24_sarah.pdf) (Sarah Gaiser, 3/24/2025)
The cuts are listed below.
#### Pre-Cut:
* Event has at least 1 electron and 1 positron: $N_{e^{-}}>0$ && $N_{e^{+}}>0$ 
#### Track Cuts:
$$0.4\mathrm{\ GeV} < p_{e^-} < 2.9\mathrm{\ GeV}$$
$$0.4\mathrm{\ GeV} < p_{e^+} < 4.0\mathrm{\ GeV}$$
$$\Delta (t_{trk, e^-}, t_{clu, e^+} < 10 \mathrm{\ ns}$$
$$\Delta(t_{trk, e^+}, t_{clu, e^+} < 5 \mathrm{\ ns}$$
$$ \chi^2_{trk} < 20 $$
$$ N_{2D hits} \ge 9$$

#### Vertex Cuts
$$p_{vtx} < 4.0\mathrm{\ GeV}$$
$$ \chi^2_{vtx} < 20 $$

#### Cluster Cuts
$$ 0.2\mathrm{\ GeV} < E_{clu, e^+} < 4.0\mathrm{\ GeV}$$
* Require single3 trigger for the event.

#### Cut Flow (Sarah 5/6/2025)
<img src="file:/System/Volumes/Data/data/HPS/Analyses/TriTrig/images/Sarah_cuflow.png" alt="Cutflow from Sarah's talk (5/6/2025)" width="1500" />

In [5]:
dfx0 = df\
    .Define("ucv0_idx","vector<int> out; for(int i=0;i<v0_type.size();++i) if(v0_type[i]==1) out.push_back(i); return out;") \
    .Define("pure_ele_idx","vector<int> out; for(int i=0;i<part_pdg.size();++i) if(part_pdg[i]==11) out.push_back(i); return out;") \
    .Define("puse_pos_idx","vector<int> out; for(int i=0;i<part_pdg.size();++i) if(part_pdg[i]==-11) out.push_back(i); return out;") \
    .Define("ele_idx","vector<int> out; for(int i: ucv0_idx) out.push_back(v0_em_part[i]); return out;") \
    .Define("pos_idx","vector<int> out; for(int i: ucv0_idx) out.push_back(v0_ep_part[i]); return out;") \
    .Filter("ele_idx.size() > 0 && pos_idx.size() > 0")

dfx = dfx0.Redefine("pos_idx","vector<int> out; for(int i: ucv0_idx) if(v0_ep_clus[i]>=0) out.push_back(v0_ep_part[i]); return out;") \
    .Filter("ele_idx.size() > 0 && pos_idx.size() > 0")
    
    # Note: The trigger bits are not set for MC data so will all show zero.
h_trigger_bits = dfx.Define("trigbits","vector<int> out; for(int i=0;i<20;++i) if(trigger & (1U<<i)) out.push_back(i); return out;").Histo1D(("h_trigger_bits","Trigger Bits",21,-0.5,20.5),"trigbits")
h_trigger = dfx.Histo1D(("h_trigger","Trigger",12,-1.5,10.5),"trigger")
#
dfx = dfx.Define("part_p","vector<double> out; for(int i=0;i<part_pdg.size();++i){out.push_back(sqrt(part_px[i]*part_px[i]+part_py[i]*part_py[i]+part_pz[i]*part_pz[i]));} return out;")


In [6]:
h_n_ele0 = dfx0.Define("n_ele","return ele_idx.size();").Histo1D(("h_n_ele","N e^{-};N",11,-0.5,10.5),"n_ele")
h_n_pos0 = dfx0.Define("n_pos","return pos_idx.size();").Histo1D(("h_n_ele","N e^{+};N",11,-0.5,10.5),"n_pos")

In [7]:
def Define_Histograms(dfx_in, num):
    out = {}
    out["vertex_type"] = dfx_in.Histo1D((f"vertex_type_{num}","Vertex Type",16,-0.5,15.5),"v0_type")
    out["n_v0"] = dfx_in.Define("n_v0","return ucv0_idx.size()").Histo1D((f"n_v0_{num}","N Unconstrained Vertex;N",11,-0.5,10.5),"n_v0")
    out["part_ele_p"] = dfx_in.Define("part_ele_p","vector<double> out; for(int i: ucv0_idx) out.push_back(part_p[v0_em_part[i]]); return out;").Histo1D((f"part_ele_p_{num}","Electron Momentum;p [GeV]",500,-0.1,5.),"part_ele_p")
    out["part_pos_p"] = dfx_in.Define("part_pos_p","vector<double> out; for(int i: ucv0_idx) out.push_back(part_p[v0_ep_part[i]]); return out;").Histo1D((f"part_pos_p_{num}","Positron Momentum;p [GeV]",500,-0.1,5.),"part_pos_p")
    out["pos_clus_time"] = dfx_in.Define("pos_clus_time",f"vector<double> out; for(int i: ucv0_idx) out.push_back(v0_ep_clus_time[i]+{Ecal_Cluster_Time_offset}); return out; ").Histo1D((f"pos_clus_time_{num}","Positron ECAL cluster time; t[ns]",500,-60,-20),"pos_clus_time")
    out["pos_track_time"] = dfx_in.Define("pos_track_time","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_ep_track_time[i]); return out; ").Histo1D((f"pos_track_time_{num}","Positron Track time; t[ns]",500,-60,-20),"pos_track_time")
    out["ele_track_time"] = dfx_in.Define("ele_track_time","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_em_track_time[i]); return out; ").Histo1D((f"ele_track_time_{num}","Electron Track time; t[ns]",500,-60,-20),"ele_track_time")
    out["pos_track_clus_dt"] = dfx_in.Define("pos_track_pos_clus_dt",f"vector<double> out; for(int i: ucv0_idx) out.push_back(v0_ep_clus_time[i]+{Ecal_Cluster_Time_offset} - v0_ep_track_time[i]); return out; ").Histo1D((f"pos_track_clus_df_{num}","Positron track - e+ cluster dt; dt[ns]",500,-20,20),"pos_track_pos_clus_dt")
    out["ele_track_clus_dt"] = dfx_in.Define("ele_track_pos_clus_dt",f"vector<double> out; for(int i: ucv0_idx) out.push_back(v0_ep_clus_time[i]+{Ecal_Cluster_Time_offset} - v0_em_track_time[i]); return out; ").Histo1D((f"ele_track_clus_df_{num}","Electron track - e+ cluster dt; dt[ns]",500,-20,20),"ele_track_pos_clus_dt")
    out["ele_track_chi"] = dfx_in.Define("ele_track_chi","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_em_chi2[i]); return out;").Histo1D((f"ele_track_chi_{num}","Electron track #chi^{2};#chi^{2}",500,-1,80),"ele_track_chi")
    out["pos_track_chi"] = dfx_in.Define("pos_track_chi","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_ep_chi2[i]); return out;").Histo1D((f"pos_track_chi_{num}","Positron track #chi^{2};#chi^{2}",500,-1,80),"pos_track_chi")
    out["pos_n_hits"] = dfx_in.Define("pos_track_n_hits","vector<int> out; for(int i: ucv0_idx) out.push_back(v0_ep_track_nhit[i]); return out;").Histo1D((f"pos_n_hits_{num}","Positron N hits on track;N",16,-0.5,15.5),"pos_track_n_hits")
    out["ele_n_hits"] = dfx_in.Define("ele_track_n_hits","vector<int> out; for(int i: ucv0_idx) out.push_back(v0_ep_track_nhit[i]); return out;").Histo1D((f"h_ele_n_hits_{num}","Electron N hits on track;N",16,-0.5,15.5),"ele_track_n_hits")
    out["pos_clus_energy"] = dfx_in.Define("pos_clus_energy","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_ep_clus_energy[i]); return out; ").Histo1D((f"pos_clus_energy_{num}","Positron Cluster Energy; E[GeV]",500,-0.1,5.),"pos_clus_energy")
    out["ele_clus_energy"] = dfx_in.Define("ele_clus_energy","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_em_clus_energy[i]); return out; ").Histo1D((f"ele_clus_energy_{num}","Electron Cluster Energy; E[GeV]",500,-0.1,5.),"ele_clus_energy")
    out["vertex_chi2"] = dfx_in.Define("v0_type1_chi2","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_vertex_chi2[i]); return out;").Histo1D((f"vertex_chi2_{num}","UC Vertex #chi^{2}; #chi^{2}",500,0.,70.),"v0_type1_chi2")
    out["vertex_p"] = dfx_in.Define("v0_type1_p","vector<double> out; for(int i: ucv0_idx) out.push_back(sqrt(v0_px[i]*v0_px[i]+v0_py[i]*v0_py[i]+v0_pz[i]*v0_pz[i])); return out;").Histo1D((f"vertex_p_{num}","UC Vertex p; p[GeV]",500,0.,5.),"v0_type1_p")
    # 
    return out

In [18]:
# [dataframe, filter on ucv0_idx, name, histogram collection]
idx_dataframe = 0
idx_filter = 1
idx_name = 2
idx_histo = 3
filter_set = []
filter_set.append([None, "vector<int> out;for(int i: ucv0_idx) if(v0_em_p[i]>0.4 && v0_em_p[i] < 2.9) out.push_back(i); return out;", "0.4 < P_{e^{-}} < 2.9 GeV", None])
filter_set.append([None, "vector<int> out;for(int i: ucv0_idx) if(v0_ep_p[i]>0.4 && v0_ep_p[i] < 4.0) out.push_back(i); return out;", "0.4 < P_{e^{+}} < 4.0 GeV", None])
filter_set.append([None, "vector<int> out;for(int i: ucv0_idx) if(v0_em_track_nhit[i]>=9 && v0_ep_track_nhit[i]>=9) out.push_back(i); return out;", "N hits on track >= 9", None])
filter_set.append([None, "vector<int> out; for(int i: ucv0_idx) if(v0_em_chi2[i]<20 && v0_ep_chi2[i]<20) out.push_back(i); return out;", "Track #chi^2 < 20", None])
filter_set.append([None, f"vector<int> out; for(int i: ucv0_idx) if(abs(v0_ep_clus_time[i]+{Ecal_Cluster_Time_offset} - v0_ep_track_time[i]) < 5. && abs(v0_ep_clus_time[i]+{Ecal_Cluster_Time_offset} - v0_em_track_time[i]) < 5.) out.push_back(i); return out; ", " | track - e^{+} clus dt | < 5 ns", None])
# filter_set.append([None, f"vector<int> out; for(int i: ucv0_idx) if() out.push_back(i); return out; ", " |e^{-} trk- e^{+} clus dt | < 5 ns", None])
filter_set.append([None, f"vector<int> out; for(int i: ucv0_idx) if(sqrt(v0_px[i]*v0_px[i]+v0_py[i]*v0_py[i]+v0_pz[i]*v0_pz[i] ) < 4.0) out.push_back(i); return out; ", "Vertex p < 4.0", None])
filter_set.append([None, f"vector<int> out; for(int i: ucv0_idx) if(v0_vertex_chi2[i] < 20.) out.push_back(i); return out; ", "Vertex #chi^{2} < 20.", None])
filter_set.append([None, "vector<int> out;for(int i: ucv0_idx) if(v0_ep_clus_energy[i] > 0.2 && v0_ep_clus_energy[i]< 4.0) out.push_back(i); return out; ", "Cluster e+ Energy > 0.2 and < 4.0", None])



In [19]:
# Update this for ucv0_idx instead ele_idx and pos_idx.
previous_df = dfx
for i in range(len(filter_set)):
    if filter_set[i][idx_filter] is not None:
        previous_df = previous_df.Redefine("ucv0_idx",filter_set[i][idx_filter])
    filter_set[i][idx_dataframe] = previous_df.Filter("ucv0_idx.size()>0",filter_set[i][idx_name])
    filter_set[i][idx_histo] = Define_Histograms(filter_set[i][idx_dataframe],i)
    previous_df = filter_set[i][idx_dataframe]

In [20]:
print(f" N {'Filter':>45s} {'Count':>8s}")
N_m3 = df.Count()
N_m2 = dfx0.Count()
N_m1 = dfx.Count()
print(f"{-3:2d} {'No cuts':>45s} {N_m3.GetValue():8d}")
print(f"{-2:2d} {'N_e^{-}>0  N_e^{+}>0':>45s} {N_m2.GetValue():8d}")
print(f"{-1:2d} {'N_e^{+} w. cluster>0':>45s} {N_m1.GetValue():8d}")
N = []
for i in range(len(filter_set)):
    N.append(filter_set[i][idx_dataframe].Count())

for i in range(len(filter_set)):
    print(f"{i:2d} {filter_set[i][idx_name]:>45s} {N[i].GetValue():8d}  {filter_set[i][idx_histo]['n_v0'].GetEntries():9.1f}")


 N                                        Filter    Count
-3                                       No cuts   209614
-2                          N_e^{-}>0  N_e^{+}>0    85961
-1                          N_e^{+} w. cluster>0    82127
 0                     0.4 < P_{e^{-}} < 2.9 GeV    58793    58793.0
 1                     0.4 < P_{e^{+}} < 4.0 GeV    58793    58793.0
 2                          N hits on track >= 9    46482    46482.0
 3                             Track #chi^2 < 20    32816    32816.0
 4              | track - e^{+} clus dt | < 5 ns    30494    30494.0
 5                                Vertex p < 4.0    30426    30426.0
 6                         Vertex #chi^{2} < 20.    30401    30401.0
 7             Cluster e+ Energy > 0.2 and < 4.0    30400    30400.0


In [22]:
his_names = ["vertex_type","n_v0","part_ele_p", "part_pos_p", 
              #"pos_clus_time","pos_clus_time", 
              #"ele_track_time", "pos_track_time", 
              "ele_track_clus_dt", "pos_track_clus_dt", 
              "ele_n_hits","pos_n_hits", 
              "ele_track_chi","pos_track_chi",
              "pos_clus_energy","ele_clus_energy", 
              "vertex_chi2","vertex_p"
             ]
colors = [R.kBlue,R.kBlue-5,R.kCyan,R.kGreen, R.kGreen+2,R.kYellow,R.kOrange,R.kRed]
thickness = [3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
cc = R.TCanvas("cc","cc",1000,300*len(his_names)//2+2*len(his_names)%2)
cc.Divide(2,len(his_names)//2+2*len(his_names)%2)


for n in range(len(his_names)):
    pad = cc.cd(n+1)
    pad.SetLogy()
    
    if n == 0:
        leg1 = R.TLegend(0.65,0.4,0.99,0.75)
    for i in range(len(filter_set)):
        filter_set[i][idx_histo][his_names[n]].SetLineColor(colors[i])
        filter_set[i][idx_histo][his_names[n]].SetLineWidth(thickness[i])
        filter_set[i][idx_histo][his_names[n]].Draw("same")
        if n == 0:
            leg1.AddEntry(filter_set[i][idx_histo]["n_v0"].GetPtr(),filter_set[i][idx_name])
    if n == 0:
        leg1.Draw()
cc.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc


In [25]:
cc = R.TCanvas("cc1","cc",800,600)
cc.SetLogy()
#h_tmp = dfx.Histo1D(("h_v0_type","Vertex type",16,-0.5,15.5),"v0_type")
#h_tmp = dfx.Define("v0_type1_chi2","vector<double> out; for(int i: ucv0_idx) out.push_back(v0_vertex_chi2[i]); return out;").Histo1D((f"vertex_chi2","UC Vertex #chi^{2}; #chi^{2}",500,0.,70.),"v0_type1_chi2")
h_tmp = dfx.Define("v0_type1_p","vector<double> out; for(int i: ucv0_idx) out.push_back(sqrt(v0_px[i]*v0_px[i]+v0_py[i]*v0_py[i]+v0_pz[i]*v0_pz[i])); return out;").Histo1D((f"vertex_p","UC Vertex p; p[GeV]",500,0.,5.),"v0_type1_p")
h_tmp.Draw()

cc.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc1
